## 1. Pre-trained neural networks

Researchers have built accurate image recognition systems. So instead of training our own neural network from scratch, we could reuse an existing neural network design as a starting point in projects. Some popular neural network designs that we can reuse:

- **VGG** is a deep network with either 16 or 19 layers, *University of Oxford* (2014) 
- **ResNet-50** is a 50 layer neural network, *Microsoft* (2015)
- **Inception v3** is another design, *Google* (2015) 

These pre-trained neural networks are included with Keras. The models are all trained to recognize images from the ImageNet data set.

In [3]:
import numpy as np
from keras.preprocessing import image
from keras.applications import vgg16

Using TensorFlow backend.


In [ ]:
# Load Keras' VGG16 model that was pre-trained against the ImageNet database
model = vgg16.VGG16()

In [4]:
# Load the image file, resizing it to 224x224 pixels (required by this model)
img = image.load_img("bay.jpg", target_size=(224, 224))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Add a fourth dimension (since Keras expects a list of images)
x = np.expand_dims(x, axis=0)

# Normalize the input image's pixel values to the range used when training the neural network
x = vgg16.preprocess_input(x)

# Run the image through the deep neural network to make a prediction
predictions = model.predict(x)

# Look up the names of the predicted classes. Index zero is the results for the first image.
predicted_classes = vgg16.decode_predictions(predictions)

print("Top predictions for this image:")

for imagenet_id, name, likelihood in predicted_classes[0]:
    print("Prediction: {} - {:2f}".format(name, likelihood))

Top predictions for this image:
Prediction: seashore - 0.395213
Prediction: promontory - 0.326128
Prediction: lakeside - 0.119613
Prediction: breakwater - 0.062801
Prediction: sandbar - 0.045267


## 2. Transfer learning (another alternative to training a new neural network)

In reality, you almost never need to train the neural network from scratch. Instead, we can use transfer learning to reuse an existing neural network and adapt it to a new problem. Transfer learning is where you take a model trained on one set of data and then use the knowledge it learned to give it a headstart when solving a new problem.

The basic idea is that neural networks learn to detect simple patterns in the top layer, and then the next layer uses that information to look for slightly more complex patterns and so on, down through all the convolutional layers. But the final layer of the neural network is a densely connected layer that uses the information from the convolutional layers to decide which object is in the image. 

With transfer learning, we start with a neural network that's already been trained to recognize objects from a large dataset. To reuse this neural network with new data, we can simply slice off the last layer. We will keep all the layers that detect patterns, but remove the part that maps those patterns to specific objects and ``call this pre-trained neural network a feature extractor`` because we're using it to extract training features from images. Then, we create a new neural network to replace the last layer in the original network. This is the only part that we will have to train ourselves. 

In [1]:
from pathlib import Path
import numpy as np
import joblib

### Feature extraction

In [5]:
# Path to folders with training data
dog_path = Path("training_data") / "dogs2"
not_dog_path = Path("training_data") / "not_dogs2"

images = []
labels = []

# Load all the not-dog images
for img in not_dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'not dog' image, the expected value should be 0
    labels.append(0)

# Load all the dog images
for img in dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'dog' image, the expected value should be 1
    labels.append(1)

# Create a single numpy array with all the images we loaded
x_train = np.array(images)

# Also convert the labels to a numpy array
y_train = np.array(labels)

# Normalize image data to 0-to-1 range
x_train = vgg16.preprocess_input(x_train)

# Load a pre-trained neural network to use as a feature extractor
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Extract features for each image (all in one pass)
features_x = pretrained_nn.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")

58892288/58889256 [==============================] - 10s 0us/step


['y_train.dat']

### Training with extracted features

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [7]:
# Load data set
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")

# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

# Train the model
model.fit(
    x_train,
    y_train,
    epochs=10,
    shuffle=True
)

# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
58/58 [==============================] - 1s 11ms/step - loss: 15.7117 - accuracy: 0.4483
Epoch 2/10
58/58 [==============================] - 0s 256us/step - loss: 1.9552 - accuracy: 0.8621
Epoch 3/10
58/58 [==============================] - 0s 205us/step - loss: 0.2158 - accuracy: 0.9655
Epoch 4/10
58/58 [==============================] - 0s 189us/step - loss: 0.1633 - accuracy: 0.9828
Epoch 5/10
58/58 [==============================] - 0s 190us/step - loss: 0.0268 - accuracy: 0.9828
Epoch 6/10
58/58 [==============================] - 0s 208us/step - loss: 0.0399 - accuracy: 0.9828
Epoch 7/10
58/58 [==============================] - 0s 199us/step - loss: 4.7841e-10 - accuracy: 1.0000
Epoch 8/10
58/58 [==============================] - 0s 269us/step - loss: 6.3526e-08 - accuracy: 1.0000
Epoch 9/10
58/58 [==============================] - 0s 188us/step - loss: 3.0882e-13 - accuracy: 1

Note that it took ``so little time to train`` the model.

### Making predictions

In [8]:
from keras.models import model_from_json

In [11]:
# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model_weights.h5")

# Testing with two images
IMGS = ["dog.png", "not_dog.png"]
for c in IMGS:
    # Load an image file to test, resizing it to 64x64 pixels (as required by this model)
    img = image.load_img(c, target_size=(64, 64))

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add a forth dimension to the image (since Keras expects a bunch of images, not a single image)
    images = np.expand_dims(image_array, axis=0)

    # Normalize the data
    images = vgg16.preprocess_input(images)

    # Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
    feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
    features = feature_extraction_model.predict(images)

    # Given the extracted features, make a final prediction using our own model
    results = model.predict(features)

    # Since we are only testing one image with possible class, we only need to check the first result's first element
    single_result = results[0][0]

    # Print the result
    print("Likelihood that this image contains a dog: {} %".format(int(single_result * 100)))

Likelihood that this image contains a dog: 100 %
Likelihood that this image contains a dog: 0 %
